In [ ]:
!pip install tensorflow

In [1]:
import os
import numpy as np
from PIL import Image
from mtcnn import MTCNN
import cv2
import dlib
from imgaug import augmenters as iaa

ModuleNotFoundError: No module named 'tensorflow'